In [6]:
import pickle
from multiprocessing.pool import Pool

import imdb
import numpy as np
import pandas as pd

with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def get_country(nconst):
    ia = imdb.IMDb()
    person = ia.get_person(nconst[2:])
    country = person.data["birth info"]["birth place"].split(",")[-1].strip()
    print(country)
    return country

def get_countries(df):
    df["birth_country"] = df["nconst"].apply(get_country)
    return df

def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

df = parallelize_dataframe(df, get_countries)

print(df)

UsageError: Line magic function `%%timeit` not found.
